In [2]:
import pandas as pd
import random
import csv
from collections import defaultdict, Counter
import math

C:\Users\user\AppData\Local\Temp/ipykernel_20312/1410569587.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
NOTES = ["C", "C#", "D", "Eb", "E", "F", "F#", "G", "Ab", "A", "Bb", "B"]
REST = {
            4: "whole rest",
            2: "half rest",
            1: "Quarter rest",
            0.5: "eighth rest",
            0.25: "Sixteenth rest",
            0: "no rest",
        }
TEMPO = [80, 100, 120, 140, 160]
RTYTHM = {4:"whole", 2:"white", 1:"black", 0.5:"half", 0.25:"Quater"}
OCTAVES = list(range(11))
NOTES_IN_OCTAVE = len(NOTES)

errors = {"notes": "Bad input, please refer this spec-\n"}

array_of_notes = NOTES  # ตั้งว่าจะเอาโน๊ตอะไรบ้าง

array_of_note_numbers = []
array_temp = []
for note in range(len(array_of_notes)):
    array_temp = []
    OCTAVE = 4  # เปลี่ยน octive ตรงนี้
    array_of_note_numbers.append(note)
print(array_of_note_numbers)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [4]:
def swap_accidentals(note):
    if note == "Db":
        return "C#"
    if note == "D#":
        return "Eb"
    if note == "E#":
        return "F"
    if note == "Gb":
        return "F#"
    if note == "G#":
        return "Ab"
    if note == "A#":
        return "Bb"
    if note == "B#":
        return "C"

    return note


def note_to_number(note: str, octave: int) -> int:
    note = swap_accidentals(note)
    assert note in NOTES, errors["notes"]
    assert octave in OCTAVES, errors["notes"]

    note = NOTES.index(note)
    note += NOTES_IN_OCTAVE * octave

    assert 0 <= note <= 127, errors["notes"]

    return note


# 1.กรณีไม่มีตัวหยุด

1.1 โน๊ตเท่ากัน

In [5]:
def create_csv(row, writer, rhythm=[2, 2, 2], rest=[0, 0, 0], tempo=120):
    label = eval(row['label'])
    label = [swap_accidentals(la) for la in label]
    label_str = "".join(label)
    label_int = ''.join([str(NOTES.index(note)) for note in label])
    all_rhythm = [RTYTHM[rhy] for rhy in rhythm]
    all_rest = [REST[r] for r in rest]

    writer.writerow([f"wav_{label_str+label_int}.wav", label, tempo, all_rhythm, all_rest, len(label)])


In [20]:
# 1.1.1
data_train = pd.read_csv("../../data/ref_3_note_600_train.csv")
# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
csv_file = "../../data/exp1-1/1-1-1/file_names_exp1-1-1_train.csv"
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest", "num of pitch"])
    data_train.apply(lambda row: create_csv(row, writer), axis=1)


data_valid = pd.read_csv("../../data/ref_3_note_200_validate.csv")
csv_file = "../../data/exp1-1/1-1-1/file_names_exp1-1-1_validate.csv"
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest", "num of pitch"])
    data_valid.apply(lambda row: create_csv(row, writer), axis=1)


data_test = pd.read_csv("../../data/ref_3_note_200_test.csv")
csv_file = "../../data/exp1-1/1-1-1/file_names_exp1-1-1_test.csv"
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest", "num of pitch"])
    data_valid.apply(lambda row: create_csv(row, writer), axis=1)


In [21]:
# 1.1.2
def balanced_tempo(values, sample_size):
    # สร้างดิกชันนารีเก็บจำนวนครั้งที่แต่ละค่าใน values ถูกสุ่มมาแล้ว
    count = defaultdict(int)
    # จำนวนครั้งที่ควรสุ่มได้เท่า ๆ กัน
    target_count = sample_size // len(values)
    
    tempos = []
    
    for _ in range(sample_size):
        while True:
            # สุ่มค่า
            value = random.choice(values)
            # ตรวจสอบว่าค่าที่สุ่มมานั้นยังไม่เกินจำนวนเป้าหมายที่ต้องการ
            if count[value] < target_count:
                tempos.append(value)
                count[value] += 1
                break
            # กรณีที่เกิน target_count แล้ว ให้สุ่มใหม่
    return tempos

# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
data_train = pd.read_csv("../../data/ref_3_note_600_train.csv")
csv_file = "../../data/exp1-1/1-1-2/file_names_exp1-1-2_train.csv"
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest", "num of pitch"])
    tempos = balanced_tempo(TEMPO, 600)
    tempos = pd.DataFrame(tempos, columns=['tempo'])
    data_train = pd.concat([data_train, tempos], axis=1)
    data_train.apply(lambda row: create_csv(row, writer, tempo=row['tempo']), axis=1)


data_valid = pd.read_csv("../../data/ref_3_note_200_validate.csv")
csv_file = "../../data/exp1-1/1-1-2/file_names_exp1-1-2_validate.csv"
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest", "num of pitch"])
    tempos = balanced_tempo(TEMPO, 200)
    tempos = pd.DataFrame(tempos, columns=['tempo'])
    data_valid = pd.concat([data_valid, tempos], axis=1)
    data_valid.apply(lambda row: create_csv(row, writer, tempo=row['tempo']), axis=1)


data_test = pd.read_csv("../../data/ref_3_note_200_test.csv")
csv_file = "../../data/exp1-1/1-1-2/file_names_exp1-1-2_test.csv"
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest", "num of pitch"])
    tempos = balanced_tempo(TEMPO, 200)
    tempos = pd.DataFrame(tempos, columns=['tempo'])
    data_test = pd.concat([data_test, tempos], axis=1)
    data_test.apply(lambda row: create_csv(row, writer, tempo=row['tempo']), axis=1)


In [22]:
# 1.1.3
def balanced_rhythm(note_count, sequence_count, tolerance):
    sequences = []
    note_frequency = [defaultdict(int) for _ in range(note_count)]
    samples_per_note = math.floor(sequence_count / len(RTYTHM))

    # สร้างลำดับโน๊ตให้ครบตามจำนวนที่ต้องการ
    while len(sequences) < sequence_count:
        available_rest = []
        for position in range(note_count):
            available_note = []
            available_note = [
                note
                for note in list(RTYTHM.keys())
                if note_frequency[position][note] < samples_per_note + tolerance
            ]
            available_rest.append(available_note)

        sequence = [
            random.choice(note_each_position) for note_each_position in available_rest
        ]
        # print(sequence, available_rest)

        if all(
            note_frequency[i][note] < samples_per_note + tolerance
            for i, note in enumerate(sequence)
        ):
            sequences.append(sequence)
            for i, note in enumerate(sequence):
                note_frequency[i][note] += 1

    return list(sequences)


# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
data_train = pd.read_csv("../../data/ref_3_note_600_train.csv")
csv_file = "../../data/exp1-1/1-1-3/file_names_exp1-1-3_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rhythm = balanced_rhythm(3, 600, 1)
    rhythm = pd.DataFrame({'rhythm': [str(item) for item in rhythm]})
    data_train = pd.concat([data_train, rhythm], axis=1)
    data_train.apply(lambda row: create_csv(row, writer, rhythm=eval(row["rhythm"])), axis=1)


data_valid = pd.read_csv("../../data/ref_3_note_200_validate.csv")
csv_file = "../../data/exp1-1/1-1-3/file_names_exp1-1-3_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rhythm = balanced_rhythm(3, 200, 1)
    rhythm = pd.DataFrame({'rhythm': [str(item) for item in rhythm]})
    data_valid = pd.concat([data_valid, rhythm], axis=1)
    data_valid.apply(lambda row: create_csv(row, writer, rhythm=eval(row["rhythm"])), axis=1)


data_test = pd.read_csv("../../data/ref_3_note_200_test.csv")
csv_file = "../../data/exp1-1/1-1-3/file_names_exp1-1-3_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rhythm = balanced_rhythm(3, 200, 1)
    rhythm = pd.DataFrame({'rhythm': [str(item) for item in rhythm]})
    data_test = pd.concat([data_test, rhythm], axis=1)
    data_test.apply(lambda row: create_csv(row, writer, rhythm=eval(row["rhythm"])), axis=1)

In [23]:
# 1.1.4
data_train_1_1_2 = pd.read_csv("../../data/exp1-1/1-1-2/file_names_exp1-1-2_train.csv")
data_train_1_1_3 = pd.read_csv("../../data/exp1-1/1-1-3/file_names_exp1-1-3_train.csv")
data_train = pd.merge(data_train_1_1_2, data_train_1_1_3, left_on='File Name', right_on='File Name')
data_train_1_1_4 = pd.DataFrame({
    'File Name': data_train['File Name'],
    'label': data_train['label_x'],
    'tempo': data_train['tempo_x'],
    'rhythm': data_train['rhythm_y'],
    'rest': data_train['rest_x'],
    'num of pitch': data_train['num of pitch_x'],
})
data_train_1_1_4.to_csv("../../data/exp1-1/1-1-4/file_names_exp1-1-4_train.csv", index=False)


data_validate_1_1_2 = pd.read_csv("../../data/exp1-1/1-1-2/file_names_exp1-1-2_validate.csv")
data_validate_1_1_3 = pd.read_csv("../../data/exp1-1/1-1-3/file_names_exp1-1-3_validate.csv")
data_validate = pd.merge(data_validate_1_1_2, data_validate_1_1_3, left_on='File Name', right_on='File Name')
data_validate_1_1_4 = pd.DataFrame({
    'File Name': data_validate['File Name'],
    'label': data_validate['label_x'],
    'tempo': data_validate['tempo_x'],
    'rhythm': data_validate['rhythm_y'],
    'rest': data_validate['rest_x'],
    'num of pitch': data_validate['num of pitch_x'],
})
data_validate_1_1_4.to_csv("../../data/exp1-1/1-1-4/file_names_exp1-1-4_validate.csv", index=False)


data_test_1_1_2 = pd.read_csv("../../data/exp1-1/1-1-2/file_names_exp1-1-2_test.csv")
data_test_1_1_3 = pd.read_csv("../../data/exp1-1/1-1-3/file_names_exp1-1-3_test.csv")
data_test = pd.merge(data_test_1_1_2, data_test_1_1_3, left_on='File Name', right_on='File Name')
data_test_1_1_4 = pd.DataFrame({
    'File Name': data_test['File Name'],
    'label': data_test['label_x'],
    'tempo': data_test['tempo_x'],
    'rhythm': data_test['rhythm_y'],
    'rest': data_test['rest_x'],
    'num of pitch': data_test['num of pitch_x'],
})
data_test_1_1_4.to_csv("../../data/exp1-1/1-1-4/file_names_exp1-1-4_test.csv", index=False)

1.2 โน๊ตไม่เท่ากัน

In [24]:
# 1.2.1
data_train = pd.read_csv("../../data/ref_list_multi_note_600_train.csv")
# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
csv_file = "../../data/exp1-2/1-2-1/file_names_exp1-2-1_train.csv"
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest", "num of pitch"])
    data_train.apply(lambda row: create_csv(row, writer, rest=[0 for _ in range(len(eval(row["label"])))]), axis=1)


data_valid = pd.read_csv("../../data/ref_list_multi_note_200_validate.csv")
csv_file = "../../data/exp1-2/1-2-1/file_names_exp1-2-1_validate.csv"
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest", "num of pitch"])
    data_valid.apply(lambda row: create_csv(row, writer, rest=[0 for _ in range(len(eval(row["label"])))]), axis=1)


data_test = pd.read_csv("../../data/ref_list_multi_note_200_test.csv")
csv_file = "../../data/exp1-2/1-2-1/file_names_exp1-2-1_test.csv"
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest", "num of pitch"])
    data_valid.apply(lambda row: create_csv(row, writer, rest=[0 for _ in range(len(eval(row["label"])))]), axis=1)


In [25]:
# 1.2.2
def balanced_tempo_multi(values, sample_size):
    # สร้างดิกชันนารีเก็บจำนวนครั้งที่แต่ละค่าใน values ถูกสุ่มมาแล้ว
    count = defaultdict(int)
    # จำนวนครั้งที่ควรสุ่มได้เท่า ๆ กัน
    target_count = sample_size // len(values)
    
    tempos = []
    
    for _ in range(sample_size):
        while True:
            # สุ่มค่า
            value = random.choice(values)
            # ตรวจสอบว่าค่าที่สุ่มมานั้นยังไม่เกินจำนวนเป้าหมายที่ต้องการ
            if count[value] < target_count:
                tempos.append(value)
                count[value] += 1
                break
            # กรณีที่เกิน target_count แล้ว ให้สุ่มใหม่
    return tempos

# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
data_train = pd.read_csv("../../data/ref_list_multi_note_600_train.csv")
csv_file = "../../data/exp1-2/1-2-2/file_names_exp1-2-2_train.csv"
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest", "num of pitch"])
    tempos = []
    for i in range(3, 11):
        tempos.extend(balanced_tempo_multi(TEMPO, 75))
    tempos = pd.DataFrame(tempos, columns=['tempo'])
    data_train = pd.concat([data_train, tempos], axis=1)
    data_train.apply(lambda row: create_csv(row, writer, tempo=row['tempo'], rest=[0 for _ in range(len(eval(row["label"])))]), axis=1)


data_valid = pd.read_csv("../../data/ref_list_multi_note_200_validate.csv")
csv_file = "../../data/exp1-2/1-2-2/file_names_exp1-2-2_validate.csv"
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest", "num of pitch"])
    tempos = []
    for i in range(3, 11):
        tempos.extend(balanced_tempo_multi(TEMPO, 25))
    tempos = pd.DataFrame(tempos, columns=['tempo'])
    data_valid = pd.concat([data_valid, tempos], axis=1)
    data_valid.apply(lambda row: create_csv(row, writer, tempo=row['tempo'], rest=[0 for _ in range(len(eval(row["label"])))]), axis=1)


data_test = pd.read_csv("../../data/ref_list_multi_note_200_test.csv")
csv_file = "../../data/exp1-2/1-2-2/file_names_exp1-2-2_test.csv"
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "rest", "num of pitch"])
    tempos = []
    for i in range(3, 11):
        tempos.extend(balanced_tempo_multi(TEMPO, 25))
    tempos = pd.DataFrame(tempos, columns=['tempo'])
    data_test = pd.concat([data_test, tempos], axis=1)
    data_test.apply(lambda row: create_csv(row, writer, tempo=row['tempo'], rest=[0 for _ in range(len(eval(row["label"])))]), axis=1)


In [26]:
# 1.2.3
def balanced_rhythm_multi(note_count, sequence_count, tolerance):
    sequences = []
    note_frequency = [defaultdict(int) for _ in range(note_count)]
    samples_per_note = math.floor(sequence_count / len(RTYTHM))

    # สร้างลำดับโน๊ตให้ครบตามจำนวนที่ต้องการ
    while len(sequences) < sequence_count:
        available_rest = []
        for position in range(note_count):
            available_note = []
            available_note = [
                note
                for note in list(RTYTHM.keys())
                if note_frequency[position][note] < samples_per_note + tolerance
            ]
            available_rest.append(available_note)

        sequence = [
            random.choice(note_each_position) for note_each_position in available_rest
        ]
        # print(sequence, available_rest)

        if all(
            note_frequency[i][note] < samples_per_note + tolerance
            for i, note in enumerate(sequence)
        ):
            sequences.append(sequence)
            for i, note in enumerate(sequence):
                note_frequency[i][note] += 1

    return list(sequences)


# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
data_train = pd.read_csv("../../data/ref_list_multi_note_600_train.csv")
csv_file = "../../data/exp1-2/1-2-3/file_names_exp1-2-3_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rhythm = []
    for i in range(3, 11):
        rhythm.extend(balanced_rhythm_multi(i, 75, 1))
    rhythm = pd.DataFrame({'rhythm': [str(item) for item in rhythm]})
    data_train = pd.concat([data_train, rhythm], axis=1)
    data_train.apply(lambda row: create_csv(row, writer, rhythm=eval(row["rhythm"]), rest=[0 for _ in range(len(eval(row["label"])))]), axis=1)


data_valid = pd.read_csv("../../data/ref_list_multi_note_200_validate.csv")
csv_file = "../../data/exp1-2/1-2-3/file_names_exp1-2-3_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rhythm = []
    for i in range(3, 11):
        rhythm.extend(balanced_rhythm_multi(i, 25, 1))
    rhythm = pd.DataFrame({'rhythm': [str(item) for item in rhythm]})
    data_valid = pd.concat([data_valid, rhythm], axis=1)
    data_valid.apply(lambda row: create_csv(row, writer, rhythm=eval(row["rhythm"]), rest=[0 for _ in range(len(eval(row["label"])))]), axis=1)


data_test = pd.read_csv("../../data/ref_list_multi_note_200_test.csv")
csv_file = "../../data/exp1-2/1-2-3/file_names_exp1-2-3_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rhythm = []
    for i in range(3, 11):
        rhythm.extend(balanced_rhythm_multi(i, 25, 1))
    rhythm = pd.DataFrame({'rhythm': [str(item) for item in rhythm]})
    data_test = pd.concat([data_test, rhythm], axis=1)
    data_test.apply(lambda row: create_csv(row, writer, rhythm=eval(row["rhythm"]), rest=[0 for _ in range(len(eval(row["label"])))]), axis=1)

In [27]:
# 1.2.4
data_train_1_2_2 = pd.read_csv("../../data/exp1-2/1-2-2/file_names_exp1-2-2_train.csv")
data_train_1_2_3 = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_train.csv")
data_train = pd.merge(data_train_1_2_2, data_train_1_2_3, left_on='File Name', right_on='File Name')
data_train_1_2_4 = pd.DataFrame({
    'File Name': data_train['File Name'],
    'label': data_train['label_x'],
    'tempo': data_train['tempo_x'],
    'rhythm': data_train['rhythm_y'],
    'rest': data_train['rest_x'],
    'num of pitch': data_train['num of pitch_x'],
})
data_train_1_2_4.to_csv("../../data/exp1-2/1-2-4/file_names_exp1-2-4_train.csv", index=False)


data_validate_1_2_2 = pd.read_csv("../../data/exp1-2/1-2-2/file_names_exp1-2-2_validate.csv")
data_validate_1_2_3 = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_validate.csv")
data_validate = pd.merge(data_validate_1_2_2, data_validate_1_2_3, left_on='File Name', right_on='File Name')
data_validate_1_2_4 = pd.DataFrame({
    'File Name': data_validate['File Name'],
    'label': data_validate['label_x'],
    'tempo': data_validate['tempo_x'],
    'rhythm': data_validate['rhythm_y'],
    'rest': data_validate['rest_x'],
    'num of pitch': data_validate['num of pitch_x'],
})
data_validate_1_2_4.to_csv("../../data/exp1-2/1-2-4/file_names_exp1-2-4_validate.csv", index=False)


data_test_1_2_2 = pd.read_csv("../../data/exp1-2/1-2-2/file_names_exp1-2-2_test.csv")
data_test_1_2_3 = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_test.csv")
data_test = pd.merge(data_test_1_2_2, data_test_1_2_3, left_on='File Name', right_on='File Name')
data_test_1_2_4 = pd.DataFrame({
    'File Name': data_test['File Name'],
    'label': data_test['label_x'],
    'tempo': data_test['tempo_x'],
    'rhythm': data_test['rhythm_y'],
    'rest': data_test['rest_x'],
    'num of pitch': data_test['num of pitch_x'],
})
data_test_1_2_4.to_csv("../../data/exp1-2/1-2-4/file_names_exp1-2-4_test.csv", index=False)

# 2.มีตัวหยุดคั่นทุกโน้ต และตัวหยุดเท่ากันทุกตัว

2.1 โน๊ตเท่ากัน

In [6]:
def create_csv_2(row, writer, rhythm=[2, 2, 2], rest=[0, 0, 0], tempo=120):
    label = eval(row['label'])
    label = [swap_accidentals(la) for la in label]
    label_str = "".join(label)
    label_int = ''.join([str(NOTES.index(note)) for note in label])
    # all_rhythm = [RTYTHM[rhy] for rhy in rhythm]
    all_rest = [REST[r] for r in rest]

    writer.writerow([f"wav_{label_str+label_int}.wav", label, tempo, rhythm, all_rest, len(label)])


In [29]:
# 2.1.1
data_train = pd.read_csv("../../data/exp1-1/1-1-1/file_names_exp1-1-1_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-1/file_names_exp1-1-1_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-1/file_names_exp1-1-1_test.csv")

csv_file = "../../data/exp2-1/2-1-1/file_names_exp2-1-1_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=[2, 2, 2], tempo=row['tempo']), axis=1)


csv_file = "../../data/exp2-1/2-1-1/file_names_exp2-1-1_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=[2, 2, 2], tempo=row['tempo']), axis=1)


csv_file = "../../data/exp2-1/2-1-1/file_names_exp2-1-1_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=[2, 2, 2], tempo=row['tempo']), axis=1)


In [30]:
# 2.1.2.
data_train = pd.read_csv("../../data/exp1-1/1-1-2/file_names_exp1-1-2_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-2/file_names_exp1-1-2_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-2/file_names_exp1-1-2_test.csv")

csv_file = "../../data/exp2-1/2-1-2/file_names_exp2-1-2_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=[2, 2, 2], tempo=row['tempo']), axis=1)


csv_file = "../../data/exp2-1/2-1-2/file_names_exp2-1-2_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=[2, 2, 2], tempo=row['tempo']), axis=1)


csv_file = "../../data/exp2-1/2-1-2/file_names_exp2-1-2_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=[2, 2, 2], tempo=row['tempo']), axis=1)


In [31]:
# 2.1.3.
data_train = pd.read_csv("../../data/exp1-1/1-1-3/file_names_exp1-1-3_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-3/file_names_exp1-1-3_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-3/file_names_exp1-1-3_test.csv")

csv_file = "../../data/exp2-1/2-1-3/file_names_exp2-1-3_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=[2, 2, 2], tempo=row['tempo']), axis=1)


csv_file = "../../data/exp2-1/2-1-3/file_names_exp2-1-3_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=[2, 2, 2], tempo=row['tempo']), axis=1)


csv_file = "../../data/exp2-1/2-1-3/file_names_exp2-1-3_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=[2, 2, 2], tempo=row['tempo']), axis=1)


In [32]:
# 2.1.4
data_train = pd.read_csv("../../data/exp1-1/1-1-4/file_names_exp1-1-4_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-4/file_names_exp1-1-4_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-4/file_names_exp1-1-4_test.csv")

csv_file = "../../data/exp2-1/2-1-4/file_names_exp2-1-4_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=[2, 2, 2], tempo=row['tempo']), axis=1)


csv_file = "../../data/exp2-1/2-1-4/file_names_exp2-1-4_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=[2, 2, 2], tempo=row['tempo']), axis=1)


csv_file = "../../data/exp2-1/2-1-4/file_names_exp2-1-4_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=[2, 2, 2], tempo=row['tempo']), axis=1)


2.2 โน๊ตไม่เท่ากัน

In [33]:
# 2.2.1
data_train = pd.read_csv("../../data/exp1-2/1-2-1/file_names_exp1-2-1_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-1/file_names_exp1-2-1_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-1/file_names_exp1-2-1_test.csv")

csv_file = "../../data/exp2-2/2-2-1/file_names_exp2-2-1_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train.apply(
        lambda row: create_csv_2(
            row,
            writer,
            rhythm=eval(row["rhythm"]),
            rest=[2 for _ in range(len(eval(row["rest"])))],
            tempo=row["tempo"],
        ),
        axis=1,
    )


csv_file = "../../data/exp2-2/2-2-1/file_names_exp2-2-1_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid.apply(
        lambda row: create_csv_2(
            row,
            writer,
            rhythm=eval(row["rhythm"]),
            rest=[2 for _ in range(len(eval(row["rest"])))],
            tempo=row["tempo"],
        ),
        axis=1,
    )


csv_file = "../../data/exp2-2/2-2-1/file_names_exp2-2-1_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test.apply(
        lambda row: create_csv_2(
            row,
            writer,
            rhythm=eval(row["rhythm"]),
            rest=[2 for _ in range(len(eval(row["rest"])))],
            tempo=row["tempo"],
        ),
        axis=1,
    )

In [34]:
# 2.2.2
data_train = pd.read_csv("../../data/exp1-2/1-2-2/file_names_exp1-2-2_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-2/file_names_exp1-2-2_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-2/file_names_exp1-2-2_test.csv")

csv_file = "../../data/exp2-2/2-2-2/file_names_exp2-2-2_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train.apply(
        lambda row: create_csv_2(
            row,
            writer,
            rhythm=eval(row["rhythm"]),
            rest=[2 for _ in range(len(eval(row["rest"])))],
            tempo=row["tempo"],
        ),
        axis=1,
    )


csv_file = "../../data/exp2-2/2-2-2/file_names_exp2-2-2_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid.apply(
        lambda row: create_csv_2(
            row,
            writer,
            rhythm=eval(row["rhythm"]),
            rest=[2 for _ in range(len(eval(row["rest"])))],
            tempo=row["tempo"],
        ),
        axis=1,
    )


csv_file = "../../data/exp2-2/2-2-2/file_names_exp2-2-2_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test.apply(
        lambda row: create_csv_2(
            row,
            writer,
            rhythm=eval(row["rhythm"]),
            rest=[2 for _ in range(len(eval(row["rest"])))],
            tempo=row["tempo"],
        ),
        axis=1,
    )

In [35]:
# 2.2.3
data_train = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_test.csv")

csv_file = "../../data/exp2-2/2-2-3/file_names_exp2-2-3_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train.apply(
        lambda row: create_csv_2(
            row,
            writer,
            rhythm=eval(row["rhythm"]),
            rest=[2 for _ in range(len(eval(row["rest"])))],
            tempo=row["tempo"],
        ),
        axis=1,
    )


csv_file = "../../data/exp2-2/2-2-3/file_names_exp2-2-3_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid.apply(
        lambda row: create_csv_2(
            row,
            writer,
            rhythm=eval(row["rhythm"]),
            rest=[2 for _ in range(len(eval(row["rest"])))],
            tempo=row["tempo"],
        ),
        axis=1,
    )


csv_file = "../../data/exp2-2/2-2-3/file_names_exp2-2-3_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test.apply(
        lambda row: create_csv_2(
            row,
            writer,
            rhythm=eval(row["rhythm"]),
            rest=[2 for _ in range(len(eval(row["rest"])))],
            tempo=row["tempo"],
        ),
        axis=1,
    )

In [ ]:
# 2.2.3
data_train = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_test.csv")

csv_file = "../../data/exp2-2/2-2-3/file_names_exp2-2-3_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train.apply(
        lambda row: create_csv_2(
            row,
            writer,
            rhythm=eval(row["rhythm"]),
            rest=[2 for _ in range(len(eval(row["rest"])))],
            tempo=row["tempo"],
        ),
        axis=1,
    )


csv_file = "../../data/exp2-2/2-2-3/file_names_exp2-2-3_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid.apply(
        lambda row: create_csv_2(
            row,
            writer,
            rhythm=eval(row["rhythm"]),
            rest=[2 for _ in range(len(eval(row["rest"])))],
            tempo=row["tempo"],
        ),
        axis=1,
    )


csv_file = "../../data/exp2-2/2-2-3/file_names_exp2-2-3_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test.apply(
        lambda row: create_csv_2(
            row,
            writer,
            rhythm=eval(row["rhythm"]),
            rest=[2 for _ in range(len(eval(row["rest"])))],
            tempo=row["tempo"],
        ),
        axis=1,
    )

In [36]:
# 2.2.4
data_train = pd.read_csv("../../data/exp1-2/1-2-4/file_names_exp1-2-4_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-4/file_names_exp1-2-4_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-4/file_names_exp1-2-4_test.csv")

csv_file = "../../data/exp2-2/2-2-4/file_names_exp2-2-4_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train.apply(
        lambda row: create_csv_2(
            row,
            writer,
            rhythm=eval(row["rhythm"]),
            rest=[2 for _ in range(len(eval(row["rest"])))],
            tempo=row["tempo"],
        ),
        axis=1,
    )


csv_file = "../../data/exp2-2/2-2-4/file_names_exp2-2-4_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid.apply(
        lambda row: create_csv_2(
            row,
            writer,
            rhythm=eval(row["rhythm"]),
            rest=[2 for _ in range(len(eval(row["rest"])))],
            tempo=row["tempo"],
        ),
        axis=1,
    )


csv_file = "../../data/exp2-2/2-2-4/file_names_exp2-2-4_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test.apply(
        lambda row: create_csv_2(
            row,
            writer,
            rhythm=eval(row["rhythm"]),
            rest=[2 for _ in range(len(eval(row["rest"])))],
            tempo=row["tempo"],
        ),
        axis=1,
    )

# 3. กรณีมีตัวหยุดคั่นทุกโน๊ต และตัวหยุดไม่เท่ากัน

3.1 โน๊ตเท่ากัน

In [7]:
REST_without_0 = {
            4: "whole rest",
            2: "half rest",
            1: "Quarter rest",
            0.5: "eighth rest",
            0.25: "Sixteenth rest",
        }

In [8]:
def create_csv_3(row, writer, rhythm=[2, 2, 2], rest=[0, 0, 0], tempo=120):
    label = eval(row['label'])
    label = [swap_accidentals(la) for la in label]
    label_str = "".join(label)
    label_int = ''.join([str(NOTES.index(note)) for note in label])
    # all_rhythm = [RTYTHM[rhy] for rhy in rhythm]
    # all_rest = [REST[r] for r in rest]

    writer.writerow([f"wav_{label_str+label_int}.wav", label, tempo, rhythm, rest, len(label)])


In [9]:
def balanced_rest(note_count, sequence_count, tolerance):
    sequences = []
    note_frequency = [defaultdict(int) for _ in range(note_count)]
    samples_per_note = math.floor(sequence_count / len(REST_without_0))

    # สร้างลำดับโน๊ตให้ครบตามจำนวนที่ต้องการ
    while len(sequences) < sequence_count:
        available_rest = []
        for position in range(note_count):
            available_note = []
            available_note = [
                note
                for note in list(REST_without_0.keys())
                if note_frequency[position][note] < samples_per_note + tolerance
            ]
            available_rest.append(available_note)

        sequence = [
            random.choice(note_each_position) for note_each_position in available_rest
        ]
        # print(sequence, available_rest)

        if all(
            note_frequency[i][note] < samples_per_note + tolerance
            for i, note in enumerate(sequence)
        ):
            sequences.append(sequence)
            for i, note in enumerate(sequence):
                note_frequency[i][note] += 1

    return list(sequences)


In [41]:
# 3.1.1
data_train = pd.read_csv("../../data/exp1-1/1-1-1/file_names_exp1-1-1_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-1/file_names_exp1-1-1_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-1/file_names_exp1-1-1_test.csv")

csv_file = "../../data/exp3-1/3-1-1/file_names_exp3-1-1_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = balanced_rest(3, 600, 1)
    rest = pd.DataFrame({'rest_new': [str(item) for item in rest]})
    data_train = pd.concat([data_train, rest], axis=1)
    data_train.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest_new']), tempo=row['tempo']), axis=1)


csv_file = "../../data/exp3-1/3-1-1/file_names_exp3-1-1_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = balanced_rest(3, 200, 1)
    rest = pd.DataFrame({'rest_new': [str(item) for item in rest]})
    data_valid = pd.concat([data_valid, rest], axis=1)
    data_valid.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest_new']), tempo=row['tempo']), axis=1)


csv_file = "../../data/exp3-1/3-1-1/file_names_exp3-1-1_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = balanced_rest(3, 200, 1)
    rest = pd.DataFrame({'rest_new': [str(item) for item in rest]})
    data_test = pd.concat([data_test, rest], axis=1)
    data_test.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest_new']), tempo=row['tempo']), axis=1)


In [19]:
# 3.1.2
data_train = pd.read_csv("../../data/exp1-1/1-1-2/file_names_exp1-1-2_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-2/file_names_exp1-1-2_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-2/file_names_exp1-1-2_test.csv")

data_train_3_1_1 = pd.read_csv("../../data/exp3-1/3-1-1/file_names_exp3-1-1_train.csv")
csv_file = "../../data/exp3-1/3-1-2/file_names_exp3-1-2_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_train_3_1_1['rest']
    data_train['rest'] = data_train_3_1_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_validate_3_1_1 = pd.read_csv("../../data/exp3-1/3-1-1/file_names_exp3-1-1_validate.csv")
csv_file = "../../data/exp3-1/3-1-2/file_names_exp3-1-2_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_validate_3_1_1['rest']
    data_valid['rest'] = data_validate_3_1_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_test_3_1_1 = pd.read_csv("../../data/exp3-1/3-1-1/file_names_exp3-1-1_test.csv")
csv_file = "../../data/exp3-1/3-1-2/file_names_exp3-1-2_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_test_3_1_1['rest']
    data_test['rest'] = data_test_3_1_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)
# data_train

In [20]:
# 3.1.3
data_train = pd.read_csv("../../data/exp1-1/1-1-3/file_names_exp1-1-3_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-3/file_names_exp1-1-3_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-3/file_names_exp1-1-3_test.csv")

data_train_3_1_1 = pd.read_csv("../../data/exp3-1/3-1-1/file_names_exp3-1-1_train.csv")
csv_file = "../../data/exp3-1/3-1-3/file_names_exp3-1-3_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_train_3_1_1['rest']
    data_train['rest'] = data_train_3_1_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_validate_3_1_1 = pd.read_csv("../../data/exp3-1/3-1-1/file_names_exp3-1-1_validate.csv")
csv_file = "../../data/exp3-1/3-1-3/file_names_exp3-1-3_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_validate_3_1_1['rest']
    data_valid['rest'] = data_validate_3_1_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_test_3_1_1 = pd.read_csv("../../data/exp3-1/3-1-1/file_names_exp3-1-1_test.csv")
csv_file = "../../data/exp3-1/3-1-3/file_names_exp3-1-3_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_test_3_1_1['rest']
    data_test['rest'] = data_test_3_1_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)
# data_train

In [21]:
# 3.1.4
data_train = pd.read_csv("../../data/exp1-1/1-1-4/file_names_exp1-1-4_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-4/file_names_exp1-1-4_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-4/file_names_exp1-1-4_test.csv")

data_train_3_1_1 = pd.read_csv("../../data/exp3-1/3-1-1/file_names_exp3-1-1_train.csv")
csv_file = "../../data/exp3-1/3-1-4/file_names_exp3-1-4_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_train_3_1_1['rest']
    data_train['rest'] = data_train_3_1_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_validate_3_1_1 = pd.read_csv("../../data/exp3-1/3-1-1/file_names_exp3-1-1_validate.csv")
csv_file = "../../data/exp3-1/3-1-4/file_names_exp3-1-4_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_validate_3_1_1['rest']
    data_valid['rest'] = data_validate_3_1_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_test_3_1_1 = pd.read_csv("../../data/exp3-1/3-1-1/file_names_exp3-1-1_test.csv")
csv_file = "../../data/exp3-1/3-1-4/file_names_exp3-1-4_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_test_3_1_1['rest']
    data_test['rest'] = data_test_3_1_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)


3.2 โน๊ตไม่เท่ากัน

In [26]:
# 3.2.1
def balanced_rest_multi(note_count, sequence_count, tolerance):
    sequences = []
    note_frequency = [defaultdict(int) for _ in range(note_count)]
    samples_per_note = math.floor(sequence_count / len(REST_without_0))

    # สร้างลำดับโน๊ตให้ครบตามจำนวนที่ต้องการ
    while len(sequences) < sequence_count:
        available_rest = []
        for position in range(note_count):
            available_note = []
            available_note = [
                note
                for note in list(REST_without_0.keys())
                if note_frequency[position][note] < samples_per_note + tolerance
            ]
            available_rest.append(available_note)

        sequence = [
            random.choice(note_each_position) for note_each_position in available_rest
        ]
        # print(sequence, available_rest)

        if all(
            note_frequency[i][note] < samples_per_note + tolerance
            for i, note in enumerate(sequence)
        ):
            sequences.append(sequence)
            for i, note in enumerate(sequence):
                note_frequency[i][note] += 1

    return list(sequences)

data_train = pd.read_csv("../../data/exp1-2/1-2-1/file_names_exp1-2-1_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-1/file_names_exp1-2-1_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-1/file_names_exp1-2-1_test.csv")

# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
# data_train = pd.read_csv("../../data/ref_list_multi_note_600_train.csv")
csv_file = "../../data/exp3-2/3-2-1/file_names_exp3-2-1_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = []
    for i in range(3, 11):
        rest.extend(balanced_rest_multi(i, 75, 1))
    rest = pd.DataFrame({'rest': [str(item) for item in rest]})
    data_train['rest'] = rest['rest']
    data_train.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)


csv_file = "../../data/exp3-2/3-2-1/file_names_exp3-2-1_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = []
    for i in range(3, 11):
        rest.extend(balanced_rest_multi(i, 25, 1))
    rest = pd.DataFrame({'rest': [str(item) for item in rest]})
    data_valid['rest'] = rest['rest']
    data_valid.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)


csv_file = "../../data/exp3-2/3-2-1/file_names_exp3-2-1_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = []
    for i in range(3, 11):
        rest.extend(balanced_rest_multi(i, 25, 1))
    rest = pd.DataFrame({'rest': [str(item) for item in rest]})
    data_test['rest'] = rest['rest']
    data_test.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)


In [29]:
# 3.2.2
data_train = pd.read_csv("../../data/exp1-2/1-2-2/file_names_exp1-2-2_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-2/file_names_exp1-2-2_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-2/file_names_exp1-2-2_test.csv")

# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
data_train_3_2_1 = pd.read_csv("../../data/exp3-2/3-2-1/file_names_exp3-2-1_train.csv")
csv_file = "../../data/exp3-2/3-2-2/file_names_exp3-2-2_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train['rest'] = data_train_3_2_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_validate_3_2_1 = pd.read_csv("../../data/exp3-2/3-2-1/file_names_exp3-2-1_validate.csv")
csv_file = "../../data/exp3-2/3-2-2/file_names_exp3-2-2_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid['rest'] = data_validate_3_2_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_test_3_2_1 = pd.read_csv("../../data/exp3-2/3-2-1/file_names_exp3-2-1_test.csv")
csv_file = "../../data/exp3-2/3-2-2/file_names_exp3-2-2_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test['rest'] = data_test_3_2_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)


In [30]:
# 3.2.3
data_train = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_test.csv")

# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
data_train_3_2_1 = pd.read_csv("../../data/exp3-2/3-2-1/file_names_exp3-2-1_train.csv")
csv_file = "../../data/exp3-2/3-2-3/file_names_exp3-2-3_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train['rest'] = data_train_3_2_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_validate_3_2_1 = pd.read_csv("../../data/exp3-2/3-2-1/file_names_exp3-2-1_validate.csv")
csv_file = "../../data/exp3-2/3-2-3/file_names_exp3-2-3_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid['rest'] = data_validate_3_2_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_test_3_2_1 = pd.read_csv("../../data/exp3-2/3-2-1/file_names_exp3-2-1_test.csv")
csv_file = "../../data/exp3-2/3-2-3/file_names_exp3-2-3_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test['rest'] = data_test_3_2_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)


In [31]:
# 3.2.4
data_train = pd.read_csv("../../data/exp1-2/1-2-4/file_names_exp1-2-4_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-4/file_names_exp1-2-4_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-4/file_names_exp1-2-4_test.csv")

# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
data_train_3_2_1 = pd.read_csv("../../data/exp3-2/3-2-1/file_names_exp3-2-1_train.csv")
csv_file = "../../data/exp3-2/3-2-4/file_names_exp3-2-4_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train['rest'] = data_train_3_2_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_validate_3_2_1 = pd.read_csv("../../data/exp3-2/3-2-1/file_names_exp3-2-1_validate.csv")
csv_file = "../../data/exp3-2/3-2-4/file_names_exp3-2-4_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid['rest'] = data_validate_3_2_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_test_3_2_1 = pd.read_csv("../../data/exp3-2/3-2-1/file_names_exp3-2-1_test.csv")
csv_file = "../../data/exp3-2/3-2-4/file_names_exp3-2-4_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test['rest'] = data_test_3_2_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)


# 4. กรณีตัวหยุดไม่ได้คั่นทุกโน้ต และตัวหยุดเท่ากัน

4.1 โน๊ตเท่ากัน 

In [10]:
REST_equ = {
            # 4: "whole rest",
            2: "half rest",
            # 1: "Quarter rest",
            # 0.5: "eighth rest",
            # 0.25: "Sixteenth rest",
            0 : "no rest"
        }

def create_csv_3(row, writer, rhythm=[2, 2, 2], rest=[0, 0, 0], tempo=120):
    label = eval(row['label'])
    label = [swap_accidentals(la) for la in label]
    label_str = "".join(label)
    label_int = ''.join([str(NOTES.index(note)) for note in label])
    # all_rhythm = [RTYTHM[rhy] for rhy in rhythm]
    # all_rest = [REST[r] for r in rest]

    writer.writerow([f"wav_{label_str+label_int}.wav", label, tempo, rhythm, rest, len(label)])

def balanced_rest(note_count, sequence_count, tolerance):
    sequences = []
    note_frequency = [defaultdict(int) for _ in range(note_count)]
    samples_per_note = math.floor(sequence_count / len(REST_equ))

    # สร้างลำดับโน๊ตให้ครบตามจำนวนที่ต้องการ
    while len(sequences) < sequence_count:
        available_rest = []
        for position in range(note_count):
            available_note = []
            available_note = [
                note
                for note in list(REST_equ.keys())
                if note_frequency[position][note] < samples_per_note + tolerance
            ]
            available_rest.append(available_note)

        sequence = [
            random.choice(note_each_position) for note_each_position in available_rest
        ]
        # print(sequence, available_rest)

        if all(
            note_frequency[i][note] < samples_per_note + tolerance
            for i, note in enumerate(sequence)
        ):
            sequences.append(sequence)
            for i, note in enumerate(sequence):
                note_frequency[i][note] += 1

    return list(sequences)

# 3.1.1
data_train = pd.read_csv("../../data/exp1-1/1-1-1/file_names_exp1-1-1_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-1/file_names_exp1-1-1_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-1/file_names_exp1-1-1_test.csv")

csv_file = "../../data/exp4-1/4-1-1/file_names_exp4-1-1_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = balanced_rest(3, 600, 1)
    rest = pd.DataFrame({'rest_new': [str(item) for item in rest]})
    data_train = pd.concat([data_train, rest], axis=1)
    data_train.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest_new']), tempo=row['tempo']), axis=1)


csv_file = "../../data/exp4-1/4-1-1/file_names_exp4-1-1_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = balanced_rest(3, 200, 1)
    rest = pd.DataFrame({'rest_new': [str(item) for item in rest]})
    data_valid = pd.concat([data_valid, rest], axis=1)
    data_valid.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest_new']), tempo=row['tempo']), axis=1)


csv_file = "../../data/exp4-1/4-1-1/file_names_exp4-1-1_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = balanced_rest(3, 200, 1)
    rest = pd.DataFrame({'rest_new': [str(item) for item in rest]})
    data_test = pd.concat([data_test, rest], axis=1)
    data_test.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest_new']), tempo=row['tempo']), axis=1)

# 3.1.2
data_train = pd.read_csv("../../data/exp1-1/1-1-2/file_names_exp1-1-2_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-2/file_names_exp1-1-2_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-2/file_names_exp1-1-2_test.csv")

data_train_4_1_1 = pd.read_csv("../../data/exp4-1/4-1-1/file_names_exp4-1-1_train.csv")
csv_file = "../../data/exp4-1/4-1-2/file_names_exp4-1-2_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_train_4_1_1['rest']
    data_train['rest'] = data_train_4_1_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_validate_4_1_1 = pd.read_csv("../../data/exp4-1/4-1-1/file_names_exp4-1-1_validate.csv")
csv_file = "../../data/exp4-1/4-1-2/file_names_exp4-1-2_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_validate_4_1_1['rest']
    data_valid['rest'] = data_validate_4_1_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_test_4_1_1 = pd.read_csv("../../data/exp4-1/4-1-1/file_names_exp4-1-1_test.csv")
csv_file = "../../data/exp4-1/4-1-2/file_names_exp4-1-2_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_test_4_1_1['rest']
    data_test['rest'] = data_test_4_1_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)
# data_train

# 4.1.3
data_train = pd.read_csv("../../data/exp1-1/1-1-3/file_names_exp1-1-3_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-3/file_names_exp1-1-3_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-3/file_names_exp1-1-3_test.csv")

data_train_4_1_1 = pd.read_csv("../../data/exp4-1/4-1-1/file_names_exp4-1-1_train.csv")
csv_file = "../../data/exp4-1/4-1-3/file_names_exp4-1-3_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_train_4_1_1['rest']
    data_train['rest'] = data_train_4_1_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_validate_4_1_1 = pd.read_csv("../../data/exp4-1/4-1-1/file_names_exp4-1-1_validate.csv")
csv_file = "../../data/exp4-1/4-1-3/file_names_exp4-1-3_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_validate_4_1_1['rest']
    data_valid['rest'] = data_validate_4_1_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_test_4_1_1 = pd.read_csv("../../data/exp4-1/4-1-1/file_names_exp4-1-1_test.csv")
csv_file = "../../data/exp4-1/4-1-3/file_names_exp4-1-3_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_test_4_1_1['rest']
    data_test['rest'] = data_test_4_1_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)
# data_train

# 4.1.4
data_train = pd.read_csv("../../data/exp1-1/1-1-4/file_names_exp1-1-4_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-4/file_names_exp1-1-4_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-4/file_names_exp1-1-4_test.csv")

data_train_4_1_1 = pd.read_csv("../../data/exp4-1/4-1-1/file_names_exp4-1-1_train.csv")
csv_file = "../../data/exp4-1/4-1-4/file_names_exp4-1-4_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_train_4_1_1['rest']
    data_train['rest'] = data_train_4_1_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_validate_4_1_1 = pd.read_csv("../../data/exp4-1/4-1-1/file_names_exp4-1-1_validate.csv")
csv_file = "../../data/exp4-1/4-1-4/file_names_exp4-1-4_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_validate_4_1_1['rest']
    data_valid['rest'] = data_validate_4_1_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_test_4_1_1 = pd.read_csv("../../data/exp4-1/4-1-1/file_names_exp4-1-1_test.csv")
csv_file = "../../data/exp4-1/4-1-4/file_names_exp4-1-4_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_test_4_1_1['rest']
    data_test['rest'] = data_test_4_1_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)


4.2 โน๊ตไม่เท่ากัน

In [13]:
# 4.2.1
def balanced_rest_multi(note_count, sequence_count, tolerance):
    sequences = []
    note_frequency = [defaultdict(int) for _ in range(note_count)]
    samples_per_note = math.floor(sequence_count / len(REST_equ))

    # สร้างลำดับโน๊ตให้ครบตามจำนวนที่ต้องการ
    while len(sequences) < sequence_count:
        available_rest = []
        for position in range(note_count):
            available_note = []
            available_note = [
                note
                for note in list(REST_equ.keys())
                if note_frequency[position][note] < samples_per_note + tolerance
            ]
            available_rest.append(available_note)

        sequence = [
            random.choice(note_each_position) for note_each_position in available_rest
        ]
        # print(sequence, available_rest)

        if all(
            note_frequency[i][note] < samples_per_note + tolerance
            for i, note in enumerate(sequence)
        ):
            sequences.append(sequence)
            for i, note in enumerate(sequence):
                note_frequency[i][note] += 1

    return list(sequences)

data_train = pd.read_csv("../../data/exp1-2/1-2-1/file_names_exp1-2-1_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-1/file_names_exp1-2-1_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-1/file_names_exp1-2-1_test.csv")

# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
# data_train = pd.read_csv("../../data/ref_list_multi_note_600_train.csv")
csv_file = "../../data/exp4-2/4-2-1/file_names_exp4-2-1_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = []
    for i in range(3, 11):
        rest.extend(balanced_rest_multi(i, 75, 1))
    rest = pd.DataFrame({'rest': [str(item) for item in rest]})
    data_train['rest'] = rest['rest']
    data_train.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)


csv_file = "../../data/exp4-2/4-2-1/file_names_exp4-2-1_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = []
    for i in range(3, 11):
        rest.extend(balanced_rest_multi(i, 25, 1))
    rest = pd.DataFrame({'rest': [str(item) for item in rest]})
    data_valid['rest'] = rest['rest']
    data_valid.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)


csv_file = "../../data/exp4-2/4-2-1/file_names_exp4-2-1_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = []
    for i in range(3, 11):
        rest.extend(balanced_rest_multi(i, 25, 1))
    rest = pd.DataFrame({'rest': [str(item) for item in rest]})
    data_test['rest'] = rest['rest']
    data_test.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

# 4.2.2
data_train = pd.read_csv("../../data/exp1-2/1-2-2/file_names_exp1-2-2_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-2/file_names_exp1-2-2_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-2/file_names_exp1-2-2_test.csv")

# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
data_train_4_2_1 = pd.read_csv("../../data/exp4-2/4-2-1/file_names_exp4-2-1_train.csv")
csv_file = "../../data/exp4-2/4-2-2/file_names_exp4-2-2_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train['rest'] = data_train_4_2_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_validate_4_2_1 = pd.read_csv("../../data/exp4-2/4-2-1/file_names_exp4-2-1_validate.csv")
csv_file = "../../data/exp4-2/4-2-2/file_names_exp4-2-2_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid['rest'] = data_validate_4_2_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_test_4_2_1 = pd.read_csv("../../data/exp4-2/4-2-1/file_names_exp4-2-1_test.csv")
csv_file = "../../data/exp4-2/4-2-2/file_names_exp4-2-2_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test['rest'] = data_test_4_2_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)


# 4.2.3
data_train = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_test.csv")

# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
data_train_4_2_1 = pd.read_csv("../../data/exp4-2/4-2-1/file_names_exp4-2-1_train.csv")
csv_file = "../../data/exp4-2/4-2-3/file_names_exp4-2-3_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train['rest'] = data_train_4_2_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_validate_4_2_1 = pd.read_csv("../../data/exp4-2/4-2-1/file_names_exp4-2-1_validate.csv")
csv_file = "../../data/exp4-2/4-2-3/file_names_exp4-2-3_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid['rest'] = data_validate_4_2_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_test_4_2_1 = pd.read_csv("../../data/exp4-2/4-2-1/file_names_exp4-2-1_test.csv")
csv_file = "../../data/exp4-2/4-2-3/file_names_exp4-2-3_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test['rest'] = data_test_4_2_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)


# 4.2.4
data_train = pd.read_csv("../../data/exp1-2/1-2-4/file_names_exp1-2-4_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-4/file_names_exp1-2-4_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-4/file_names_exp1-2-4_test.csv")

# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
data_train_4_2_1 = pd.read_csv("../../data/exp4-2/4-2-1/file_names_exp4-2-1_train.csv")
csv_file = "../../data/exp4-2/4-2-4/file_names_exp4-2-4_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train['rest'] = data_train_4_2_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_validate_4_2_1 = pd.read_csv("../../data/exp4-2/4-2-1/file_names_exp4-2-1_validate.csv")
csv_file = "../../data/exp4-2/4-2-4/file_names_exp4-2-4_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid['rest'] = data_validate_4_2_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_test_4_2_1 = pd.read_csv("../../data/exp4-2/4-2-1/file_names_exp4-2-1_test.csv")
csv_file = "../../data/exp4-2/4-2-4/file_names_exp4-2-4_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test['rest'] = data_test_4_2_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)



# 5. กรณีตัวหยุดไม่ได้คั่นทุกโน้ต และตัวหยุดไม่เท่ากัน

5.1 โน๊ตเท่ากัน

In [14]:

def create_csv_3(row, writer, rhythm=[2, 2, 2], rest=[0, 0, 0], tempo=120):
    label = eval(row['label'])
    label = [swap_accidentals(la) for la in label]
    label_str = "".join(label)
    label_int = ''.join([str(NOTES.index(note)) for note in label])
    # all_rhythm = [RTYTHM[rhy] for rhy in rhythm]
    # all_rest = [REST[r] for r in rest]

    writer.writerow([f"wav_{label_str+label_int}.wav", label, tempo, rhythm, rest, len(label)])

def balanced_rest(note_count, sequence_count, tolerance):
    sequences = []
    note_frequency = [defaultdict(int) for _ in range(note_count)]
    samples_per_note = math.floor(sequence_count / len(REST))

    # สร้างลำดับโน๊ตให้ครบตามจำนวนที่ต้องการ
    while len(sequences) < sequence_count:
        available_rest = []
        for position in range(note_count):
            available_note = []
            available_note = [
                note
                for note in list(REST.keys())
                if note_frequency[position][note] < samples_per_note + tolerance
            ]
            available_rest.append(available_note)

        sequence = [
            random.choice(note_each_position) for note_each_position in available_rest
        ]
        # print(sequence, available_rest)

        if all(
            note_frequency[i][note] < samples_per_note + tolerance
            for i, note in enumerate(sequence)
        ):
            sequences.append(sequence)
            for i, note in enumerate(sequence):
                note_frequency[i][note] += 1

    return list(sequences)

# 5.1.1
data_train = pd.read_csv("../../data/exp1-1/1-1-1/file_names_exp1-1-1_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-1/file_names_exp1-1-1_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-1/file_names_exp1-1-1_test.csv")

csv_file = "../../data/exp5-1/5-1-1/file_names_exp5-1-1_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = balanced_rest(3, 600, 1)
    rest = pd.DataFrame({'rest_new': [str(item) for item in rest]})
    data_train = pd.concat([data_train, rest], axis=1)
    data_train.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest_new']), tempo=row['tempo']), axis=1)


csv_file = "../../data/exp5-1/5-1-1/file_names_exp5-1-1_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = balanced_rest(3, 200, 1)
    rest = pd.DataFrame({'rest_new': [str(item) for item in rest]})
    data_valid = pd.concat([data_valid, rest], axis=1)
    data_valid.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest_new']), tempo=row['tempo']), axis=1)


csv_file = "../../data/exp5-1/5-1-1/file_names_exp5-1-1_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = balanced_rest(3, 200, 1)
    rest = pd.DataFrame({'rest_new': [str(item) for item in rest]})
    data_test = pd.concat([data_test, rest], axis=1)
    data_test.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest_new']), tempo=row['tempo']), axis=1)

# 5.1.2
data_train = pd.read_csv("../../data/exp1-1/1-1-2/file_names_exp1-1-2_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-2/file_names_exp1-1-2_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-2/file_names_exp1-1-2_test.csv")

data_train_5_1_1 = pd.read_csv("../../data/exp5-1/5-1-1/file_names_exp5-1-1_train.csv")
csv_file = "../../data/exp5-1/5-1-2/file_names_exp5-1-2_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_train_5_1_1['rest']
    data_train['rest'] = data_train_5_1_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_validate_5_1_1 = pd.read_csv("../../data/exp5-1/5-1-1/file_names_exp5-1-1_validate.csv")
csv_file = "../../data/exp5-1/5-1-2/file_names_exp5-1-2_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_validate_5_1_1['rest']
    data_valid['rest'] = data_validate_5_1_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_test_5_1_1 = pd.read_csv("../../data/exp5-1/5-1-1/file_names_exp5-1-1_test.csv")
csv_file = "../../data/exp5-1/5-1-2/file_names_exp5-1-2_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_test_5_1_1['rest']
    data_test['rest'] = data_test_5_1_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)
# data_train

# 5.1.3
data_train = pd.read_csv("../../data/exp1-1/1-1-3/file_names_exp1-1-3_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-3/file_names_exp1-1-3_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-3/file_names_exp1-1-3_test.csv")

data_train_5_1_1 = pd.read_csv("../../data/exp5-1/5-1-1/file_names_exp5-1-1_train.csv")
csv_file = "../../data/exp5-1/5-1-3/file_names_exp5-1-3_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_train_5_1_1['rest']
    data_train['rest'] = data_train_5_1_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_validate_5_1_1 = pd.read_csv("../../data/exp5-1/5-1-1/file_names_exp5-1-1_validate.csv")
csv_file = "../../data/exp5-1/5-1-3/file_names_exp5-1-3_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_validate_5_1_1['rest']
    data_valid['rest'] = data_validate_5_1_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_test_5_1_1 = pd.read_csv("../../data/exp5-1/5-1-1/file_names_exp5-1-1_test.csv")
csv_file = "../../data/exp5-1/5-1-3/file_names_exp5-1-3_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_test_5_1_1['rest']
    data_test['rest'] = data_test_5_1_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)
# data_train

# 5.1.4
data_train = pd.read_csv("../../data/exp1-1/1-1-4/file_names_exp1-1-4_train.csv")
data_valid = pd.read_csv("../../data/exp1-1/1-1-4/file_names_exp1-1-4_validate.csv")
data_test = pd.read_csv("../../data/exp1-1/1-1-4/file_names_exp1-1-4_test.csv")

data_train_5_1_1 = pd.read_csv("../../data/exp5-1/5-1-1/file_names_exp5-1-1_train.csv")
csv_file = "../../data/exp5-1/5-1-4/file_names_exp5-1-4_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_train_5_1_1['rest']
    data_train['rest'] = data_train_5_1_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_validate_5_1_1 = pd.read_csv("../../data/exp5-1/5-1-1/file_names_exp5-1-1_validate.csv")
csv_file = "../../data/exp5-1/5-1-4/file_names_exp5-1-4_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_validate_5_1_1['rest']
    data_valid['rest'] = data_validate_5_1_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)

data_test_5_1_1 = pd.read_csv("../../data/exp5-1/5-1-1/file_names_exp5-1-1_test.csv")
csv_file = "../../data/exp5-1/5-1-4/file_names_exp5-1-4_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = data_test_4_1_1['rest']
    data_test['rest'] = data_test_4_1_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row['rhythm']), rest=eval(row['rest']), tempo=row['tempo']), axis=1)


5.2 โน๊ตไม่เท่ากัน

In [16]:
# 5.2.1
def balanced_rest_multi(note_count, sequence_count, tolerance):
    sequences = []
    note_frequency = [defaultdict(int) for _ in range(note_count)]
    samples_per_note = math.floor(sequence_count / len(REST))

    # สร้างลำดับโน๊ตให้ครบตามจำนวนที่ต้องการ
    while len(sequences) < sequence_count:
        available_rest = []
        for position in range(note_count):
            available_note = []
            available_note = [
                note
                for note in list(REST.keys())
                if note_frequency[position][note] < samples_per_note + tolerance
            ]
            available_rest.append(available_note)

        sequence = [
            random.choice(note_each_position) for note_each_position in available_rest
        ]
        # print(sequence, available_rest)

        if all(
            note_frequency[i][note] < samples_per_note + tolerance
            for i, note in enumerate(sequence)
        ):
            sequences.append(sequence)
            for i, note in enumerate(sequence):
                note_frequency[i][note] += 1

    return list(sequences)

data_train = pd.read_csv("../../data/exp1-2/1-2-1/file_names_exp1-2-1_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-1/file_names_exp1-2-1_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-1/file_names_exp1-2-1_test.csv")

# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
# data_train = pd.read_csv("../../data/ref_list_multi_note_600_train.csv")
csv_file = "../../data/exp5-2/5-2-1/file_names_exp5-2-1_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = []
    for i in range(3, 11):
        rest.extend(balanced_rest_multi(i, 75, 1))
    rest = pd.DataFrame({'rest': [str(item) for item in rest]})
    data_train['rest'] = rest['rest']
    data_train.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)


csv_file = "../../data/exp5-2/5-2-1/file_names_exp5-2-1_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = []
    for i in range(3, 11):
        rest.extend(balanced_rest_multi(i, 25, 1))
    rest = pd.DataFrame({'rest': [str(item) for item in rest]})
    data_valid['rest'] = rest['rest']
    data_valid.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)


csv_file = "../../data/exp5-2/5-2-1/file_names_exp5-2-1_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    rest = []
    for i in range(3, 11):
        rest.extend(balanced_rest_multi(i, 25, 1))
    rest = pd.DataFrame({'rest': [str(item) for item in rest]})
    data_test['rest'] = rest['rest']
    data_test.apply(lambda row: create_csv_2(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

# 5.2.2
data_train = pd.read_csv("../../data/exp1-2/1-2-2/file_names_exp1-2-2_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-2/file_names_exp1-2-2_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-2/file_names_exp1-2-2_test.csv")

# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
data_train_5_2_1 = pd.read_csv("../../data/exp5-2/5-2-1/file_names_exp5-2-1_train.csv")
csv_file = "../../data/exp5-2/5-2-2/file_names_exp5-2-2_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train['rest'] = data_train_5_2_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_validate_5_2_1 = pd.read_csv("../../data/exp5-2/5-2-1/file_names_exp5-2-1_validate.csv")
csv_file = "../../data/exp5-2/5-2-2/file_names_exp5-2-2_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid['rest'] = data_validate_5_2_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_test_5_2_1 = pd.read_csv("../../data/exp5-2/5-2-1/file_names_exp5-2-1_test.csv")
csv_file = "../../data/exp5-2/5-2-2/file_names_exp5-2-2_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test['rest'] = data_test_5_2_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)


# 5.2.3
data_train = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-3/file_names_exp1-2-3_test.csv")

# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
data_train_5_2_1 = pd.read_csv("../../data/exp5-2/5-2-1/file_names_exp5-2-1_train.csv")
csv_file = "../../data/exp5-2/5-2-3/file_names_exp5-2-3_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train['rest'] = data_train_5_2_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_validate_5_2_1 = pd.read_csv("../../data/exp5-2/5-2-1/file_names_exp5-2-1_validate.csv")
csv_file = "../../data/exp5-2/5-2-3/file_names_exp5-2-3_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid['rest'] = data_validate_5_2_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_test_5_2_1 = pd.read_csv("../../data/exp5-2/5-2-1/file_names_exp5-2-1_test.csv")
csv_file = "../../data/exp5-2/5-2-3/file_names_exp5-2-3_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test['rest'] = data_test_5_2_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)


# 5.2.4
data_train = pd.read_csv("../../data/exp1-2/1-2-4/file_names_exp1-2-4_train.csv")
data_valid = pd.read_csv("../../data/exp1-2/1-2-4/file_names_exp1-2-4_validate.csv")
data_test = pd.read_csv("../../data/exp1-2/1-2-4/file_names_exp1-2-4_test.csv")

# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
data_train_5_2_1 = pd.read_csv("../../data/exp5-2/5-2-1/file_names_exp5-2-1_train.csv")
csv_file = "../../data/exp5-2/5-2-4/file_names_exp5-2-4_train.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_train['rest'] = data_train_5_2_1['rest']
    data_train.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_validate_5_2_1 = pd.read_csv("../../data/exp5-2/5-2-1/file_names_exp5-2-1_validate.csv")
csv_file = "../../data/exp5-2/5-2-4/file_names_exp5-2-4_validate.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_valid['rest'] = data_validate_5_2_1['rest']
    data_valid.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

data_test_5_2_1 = pd.read_csv("../../data/exp5-2/5-2-1/file_names_exp5-2-1_test.csv")
csv_file = "../../data/exp5-2/5-2-4/file_names_exp5-2-4_test.csv"
with open(csv_file, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["File Name", "label", "tempo", "rhythm", "rest", "num of pitch"])
    data_test['rest'] = data_test_5_2_1['rest']
    data_test.apply(lambda row: create_csv_3(row, writer, rhythm=eval(row["rhythm"]), rest=eval(row["rest"]), tempo=row['tempo']), axis=1)

